In [2]:
import pickle
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import sklearn.manifold
import numpy as np
import pandas as pd
import re
import multiprocessing
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import collections

In [4]:
file = open("weat.txt", "r" ) 
file=file.read()

In [5]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(file.lower())

In [6]:
file2 = open("wiki.txt", "r",errors='ignore').read() #dataset

In [7]:
#Split the corpus into sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences2 = tokenizer.tokenize(file2)

In [8]:
print("Corpus is now {0} characters long".format(len(file2)))

Corpus is now 539187177 characters long


In [9]:
#convert into a list of words
#remove unnecessary, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean=re.sub('[\n]',' ',raw.lower())
    clean = re.sub("[^a-zA-Z]"," ", raw.lower())
    words = clean.split()
    #words = [ps.stem(w) for w in words if not w in stop_words]  
    words = [w for w in words if not w in stop_words] 
    return words

In [10]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences2:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))
#print(sentences[1])

In [11]:
token_count = sum([len(sentence) for sentence in sentences])
print("The corpus contains {0:,} tokens".format(token_count))

The corpus contains 48,453,527 tokens


In [12]:
def getWordList(sentences,minCount):
    wordList = []
    count = Counter()
    for sentence in sentences:
        for word in sentence:
            count[word] += 1
    for word,c in count.items():
        if c >= minCount:
            wordList.append(word)
    return wordList

In [13]:
WordList=getWordList(sentences,minCount=5) # for min frequency of a word which should be 5
#print(WordList)

In [15]:
len(WordList)

158189

In [29]:
model = Word2Vec(sentences,size=300,window=5,min_count=5,seed=102,workers=multiprocessing.cpu_count(),batch_words=1)

In [30]:
model2 = Word2Vec(sentences,size=300,window=5,min_count=5,seed=42,workers=multiprocessing.cpu_count(),batch_words=1)

In [31]:
#word_frequency
w2c = dict()
for item in model.wv.vocab:
    w2c[item]=model.wv.vocab[item].count
w2cSorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
# reverse output word and their frequency

In [32]:
def Stability2(word,similar1,similar2):    
    overlap=0
    for i in range(len(similar1)):
        for j in range(len(similar2)):
            if similar1[i][0] == similar2[j][0]:
                overlap=overlap+1
    return overlap

In [39]:
start_time = time.process_time()
with open('stability_wiki_weat.csv', 'a',errors='replace') as the_file:
    the_file.write('Word,Stability(%),Stability(%) IU,Frequency\n')
    for word in WordList:
        similar1=model.wv.most_similar(word, topn=10)
        similar2=model2.wv.most_similar(word, topn=10)
        stab = Stability2(word,similar1,similar2)
        the_file.write(word+','+str(stab*10)+','+str(stab*100/(20-stab))+','+str(w2cSorted[word])+'\n')
print (time.process_time() - start_time, "seconds")

17569.609375 seconds


In [40]:
df = pd.read_csv('stability_wiki_weat.csv')

In [41]:
df['Stability(%) IU'] = pd.to_numeric(df['Stability(%) IU'])
s = df['Stability(%) IU'].sum()
print(s)

4959061.780428033


In [42]:
def target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words):
    cos=[]
    s=0
    s1=[]
    s2=[]
    S=[]
    n=0
    for i in range(0, len(target_one_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_one_words[i][:-1]
                at1 = attribute_one_words[k][:-1]
                try:
                    cos1= model.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_one_words[i][:-1]
                at2 = attribute_two_words[k][:-1]
                try:
                    cos2= model.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s1.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
            n=n+1
    for i in range(0, len(target_two_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_two_words[i][:-1]
                at1 = attribute_one_words[k][:-1]
                try:
                    cos1= model.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_two_words[i][:-1]
                at2 = attribute_two_words[k][:-1]
                try:
                    cos2= model.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s2.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
    s=np.sum(s1)-np.sum(s2)
    stdev=np.std(S)
    print(target_one + ' vs ' + target_two  + ' , ' +attribute_one + ' vs ' + attribute_two +', d = ' + str(s/(stdev*n)))
    #print(str(s/(stdev*n)))

In [43]:
for i in range(len(raw_sentences)-30):
    words=raw_sentences[i*4].split()
    target_one = words[0][:-1]
    target_one_words = words[1:]
    words1=raw_sentences[i*4+1].split()
    target_two = words1[0][:-1]
    target_two_words = words1[1:]
    words2=raw_sentences[i*4+2].split()
    attribute_one = words2[0][:-1]
    attribute_one_words = words2[1:]
    words3 = raw_sentences[i*4+3].split()
    attribute_two = words3[0][:-1]
    attribute_two_words = words3[1:]
    
    target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words)
    

<ipython-input-42-7381828b7dd7>:15: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  cos1= model.similarity(wt, at1)
<ipython-input-42-7381828b7dd7>:28: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  cos2= model.similarity(wt, at2)
<ipython-input-42-7381828b7dd7>:46: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  cos1= model.similarity(wt, at1)
<ipython-input-42-7381828b7dd7>:59: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  cos2= model.similarity(wt, at2)


flowers vs insects , pleasant vs unpleasant, d = 1.1862986653867436
instruments vs weapons , pleasant vs unpleasant, d = 1.6614241015890672
european_american_names vs african_american_names , pleasant vs unpleasant, d = 0.9290408857617427
european_american_names vs african_american_names , pleasant vs unpleasant, d = 0.33907675519627123
european_american_names vs african_american_names , pleasant vs unpleasant, d = 0.12110345187436487
male_names vs female_names , career vs family, d = 1.9050925235393268
math vs arts , male_terms vs female_terms, d = 1.7043706938451877
science vs arts , male_terms vs female_terms, d = 1.004574865909872
mental_disease vs physical_disease , temporary vs permanent, d = 1.6480578816065627
young_people_names vs old_people_names , pleasant vs unpleasant, d = -0.7704346079420493
